In [17]:
pip install pykrx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.2/254.2 kB 14.8 MB/s eta 0:00:00


In [18]:
from pykrx import stock

In [45]:
# 시작일과 종료일 설정
start_date = "2018-01-01"
end_date = "2023-12-31"

# 코스피 지수 데이터를 가져와 데이터프레임으로 저장
kospi_index = stock.get_index_ohlcv_by_date(start_date, end_date, "1001")[['종가']].dropna()

In [65]:
kospi_index.index = money_5year_df.index
money_5year_df['KOSPI종가'] = kospi_index

In [21]:
def setMoneyRatio(df, newName, whatCol):
    df[whatCol] = pd.to_numeric(df[whatCol], errors='coerce').fillna(0)
    for i in range(1, len(df)):
        df.loc[i, newName] = calculate_ratio(df.loc[i, whatCol]- df.loc[i - 1, whatCol], df.loc[i - 1, whatCol])

In [66]:
setMoneyRatio(money_5year_df, 'KOSPI지수월간수익률', 'KOSPI종가')
setMoneyRatio(money_5year_df, '월간수익률', '종가')

In [23]:
def beta(df, target_date):
    # 날짜 형식 변환 및 1년 전 날짜 설정
    target_date = pd.to_datetime(target_date)
    one_year_ago = target_date - timedelta(days=365)

    # 기준일자 열을 datetime 형식으로 변환
    df['기준일자'] = pd.to_datetime(df['기준일자'], errors='coerce')

    # 1년 기간 필터링 및 인덱스 리셋
    filtered_df = df[(df['기준일자'] >= one_year_ago) & (df['기준일자'] <= target_date)].copy()
    filtered_df.reset_index(drop=True, inplace=True)

    # 월간 수익률 계산
    setMoneyRatio(filtered_df, 'KOSPI지수월간수익률', 'KOSPI종가')
    setMoneyRatio(filtered_df, '월간수익률', '종가')

    # 공분산과 분산 계산
    covariance = filtered_df['월간수익률'].cov(filtered_df['KOSPI지수월간수익률'])
    variance = np.var(filtered_df['KOSPI지수월간수익률'])

    # 베타 계산
    beta_value = covariance / variance
    return beta_value

In [67]:
money_5year_df['베타'] = money_5year_df['기준일자'].apply(lambda x: beta(money_5year_df, x))

/usr/local/lib/python3.10/dist-packages/pandas/core/nanops.py:1675: RuntimeWarning: Degrees of freedom <= 0 for slice
  return np.cov(a, b, ddof=ddof)[0, 1]
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)


In [68]:
# 이동평균선
money_5year_df['MA5'] = money_5year_df['종가'].rolling(window=5).mean()
money_5year_df['MA20'] = money_5year_df['종가'].rolling(window=20).mean()
money_5year_df['MA60'] = money_5year_df['종가'].rolling(window=60).mean()

In [69]:
# MACD
# ex). DataFrame.ewm(self, com=None, span=None, halflife=None, alpha=None, min_periods=0, adjust=True, ignore_na=False, axis=0)

money_5year_df['EMA12'] = money_5year_df['종가'].ewm(span=12, adjust=False).mean()
money_5year_df['EMA26'] = money_5year_df['종가'].ewm(span=26, adjust=False).mean()

money_5year_df['MACD'] = money_5year_df['EMA12'] - money_5year_df['EMA26']

In [70]:
# Signal 컬럼 및 데이터 추가
money_5year_df['Signal'] = money_5year_df['MACD'].ewm(span=9, adjust=False).mean()
money_5year_df['오실레이터'] = money_5year_df['MACD'] - money_5year_df['Signal']

In [71]:
# RSI

# 가격변동계산
money_5year_df['변동폭'] = money_5year_df['종가'].diff()

# 이득과 손실 계산
money_5year_df['이득'] = money_5year_df['변동폭'].apply(lambda x: x if x > 0 else 0)
money_5year_df['손실'] = money_5year_df['변동폭'].apply(lambda x: -x if x < 0 else 0)

# 2주 기준
money_5year_df['평균 이득'] = money_5year_df['이득'].rolling(window=14).mean()
money_5year_df['평균 손실'] = money_5year_df['손실'].rolling(window=14).mean()

# RS
money_5year_df['RS'] = money_5year_df['평균 이득'] / money_5year_df['평균 손실']

# RSI
money_5year_df['RSI'] = 100 - (100 / (1 + money_5year_df['RS']))

In [72]:
# Bollinger Bands

money_5year_df['하한선'] = money_5year_df['MA20'] - 2 * money_5year_df['종가'].rolling(window=20).std()
money_5year_df['상한선'] = money_5year_df['MA20'] + 2 * money_5year_df['종가'].rolling(window=20).std()

In [73]:
# ATR
tr1 = np.abs(money_5year_df["고가"] - money_5year_df["저가"])
tr2 = np.abs(money_5year_df["고가"] - money_5year_df["종가"].shift(1))
tr3 = np.abs(money_5year_df["저가"] - money_5year_df["종가"].shift(1))

tr = pd.DataFrame([tr1,tr2,tr3]).T.max(axis = 1)
money_5year_df["tr"] = tr
money_5year_df["atr"] = tr.rolling(14).mean()

In [74]:
money_5year_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1477 entries, 0 to 1476
Data columns (total 46 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   기준일자          1477 non-null   datetime64[ns]
 1   계열구분          1477 non-null   object        
 2   지수명           1477 non-null   object        
 3   종가            1477 non-null   float64       
 4   대비            1477 non-null   object        
 5   등락률           1477 non-null   object        
 6   시가            1477 non-null   float64       
 7   고가            1477 non-null   float64       
 8   저가            1477 non-null   float64       
 9   거래량           1477 non-null   float64       
 10  거래대금          1477 non-null   float64       
 11  상장시가총액        1477 non-null   object        
 12  전날대비거래량비율     1476 non-null   float64       
 13  전날대비거래대금비율    1476 non-null   float64       
 14  전날대비시가비율      1476 non-null   float64       
 15  전날대비고가비율      1476 non-null   float64 

In [84]:
money_5year_df_copy = money_5year_df.copy()

In [82]:
money_5year_df = money_5year_df_copy.copy()

In [85]:
# 2018년 데이터 삭제
money_5year_df_copy = money_5year_df[money_5year_df['기준일자'].dt.year != 2018]
money_5year_df = money_5year_df.reset_index(drop=True)

In [58]:
money_5year_df.reset_index(drop=True, inplace=True)

In [86]:
columns_to_drop = ['계열구분', '지수명', '종가', '대비', '등락률', '시가', '고가', '저가', '거래량', '거래대금', '상장시가총액']
money_5year_df_copy.drop(columns=columns_to_drop, errors='ignore', inplace=True)

In [87]:
columns_to_drop = ['KOSPI지수월간수익률','월간수익률','EMA12','EMA26']
money_5year_df_copy.drop(columns=columns_to_drop, errors='ignore', inplace=True)

In [89]:
columns_to_drop = ['변동폭','이득','손실','평균 이득','평균 손실','tr']
money_5year_df_copy.drop(columns=columns_to_drop, errors='ignore', inplace=True)

In [90]:
money_5year_df_copy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1233 entries, 244 to 1476
Data columns (total 25 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   기준일자        1233 non-null   datetime64[ns]
 1   전날대비거래량비율   1233 non-null   float64       
 2   전날대비거래대금비율  1233 non-null   float64       
 3   전날대비시가비율    1233 non-null   float64       
 4   전날대비고가비율    1233 non-null   float64       
 5   전날대비저가비율    1233 non-null   float64       
 6   전날대비종가비율    1233 non-null   float64       
 7   다음날종가       1233 non-null   int64         
 8   시가와고가차이     1233 non-null   float64       
 9   시가와저가차이     1233 non-null   float64       
 10  종가와고가차이     1233 non-null   float64       
 11  종가와저가차이     1233 non-null   float64       
 12  KOSPI종가     1233 non-null   float64       
 13  베타          1233 non-null   float64       
 14  MA5         1233 non-null   float64       
 15  MA20        1233 non-null   float64       
 16  MA60        1233 non-null  

In [76]:
money_5year_df_copy.set_index('기준일자', inplace=True)